# Solar

Analyze and visualize solar production.

In [ ]:
import pandas as pd
import numpy as np
import feather
import datetime

import geopandas as gpd
import PyNergy as pn

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

solaire = pd.read_feather('data/solaire.ftr')
solaire = solaire[ solaire['date'].dt.year < 2021]

sns.set_style = "seaborn-whitegrid"

color = "#333"
bck_color = "#fff"
grid_color = "#ddd"

dark_mode = True
if dark_mode:
    color = "#eee"
    bck_color = "#333"
    grid_color = "#666"


charts_colors = {
    "blue_light": "#26ffff",
    "blue": "#2696ff",
    "blue_dark": "#0759a6",

    "pink_light": "#fdc2ff",
    "pink": "#f830ff",
    "pink_dark": "#830087",

    "orange_light": "#ffdc8c",
    "orange": "#ffc130",
    "orange_dark": "#de9a00",

    "yellow_light": "#fcffcc",
    "yellow": "#ffff00",
    "yellow_dark": "#8f8f09",

    "green_light": "#abffa6",
    "green": "#33ff26",
    "green_dark": "#157a0f",

    "red_light": "#ff7878",
    "red": "#ED0000",
    "red_dark": "#a11515"   
}
    

sns.set(
    rc={
        "axes.facecolor": bck_color,
        "grid.color": grid_color,
        "grid.linestyle": "-",
        "figure.facecolor": bck_color,
        "text.color": color,
        "xtick.color": color,
        "ytick.color": color,
        "axes.labelcolor": color,
        "axes.titlecolor": color,
        "axes.grid": True,
        'axes.spines.left': True,
        'axes.spines.bottom': True,
        'axes.spines.right': False,
        'axes.spines.top': False,
        "axes.edgecolor": grid_color,
    }
)

solaire['date'] = pd.to_datetime(solaire['date'], utc=True)

solaire['annee'] = solaire['date'].dt.year
solaire['mois'] = solaire['date'].dt.month
solaire['jour'] = solaire['date'].dt.day
solaire['heure'] = solaire['date'].dt.hour

In [ ]:
solar_by_hour= solaire.groupby('heure')[['solaire', 'rayonnement_solaire']].mean()

fig, ax = plt.subplots(figsize=(10, 6))

sns.lineplot(
    data=solar_by_hour, markers=True, dashes=False, 
    ax=ax, palette=[charts_colors['blue'], charts_colors['yellow']],
    markeredgecolor=bck_color, markeredgewidth=.5, markersize=7.5)

ax.set_title('Production Solaire (gW) & Rayonnement Solaire (kW/m2) par Heures- Moyenne')
ax.set_xlabel('Heures')
ax.set_xticks(np.arange(0, 24, 1))

plt.legend(labels=['Solaire', 'Rayonnement Solaire'])

plt.savefig("outputs/03.3.1_-_Solaire_par_heure.png", dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
solar_by_month = solaire.groupby([solaire.date.dt.month_name(), 'mois'])[['solaire', 'rayonnement_solaire']].mean().reset_index()
solar_by_month.sort_values(by='mois', ascending=True, inplace=True)
solar_by_month = solar_by_month.set_index('date')
solar_by_month = solar_by_month.drop(columns=['mois'])

fig, ax = plt.subplots(figsize=(10, 6))

sns.lineplot(
    data=solar_by_month, markers=True, dashes=False, 
    ax=ax, palette=[charts_colors['blue'], charts_colors['yellow']],
    markeredgecolor=bck_color, markeredgewidth=.5, markersize=7.5)

plt.setp(ax.get_xticklabels(), rotation=45)

ax.set_title('Production Solaire (gW) & Rayonnement Solaire (kW/m2) par Mois - Moyenne')

plt.legend(labels=['Solaire', 'Rayonnement Solaire'])

plt.savefig("outputs/03.3.2_-_Solaire_par_mois.png", dpi=150, bbox_inches='tight')


plt.show()


In [ ]:

solar_by_year = solaire.groupby('annee')[['solaire', 'rayonnement_solaire']].sum()

fig, ax = plt.subplots(figsize=(10, 6))

o = sns.lineplot(
    data=solar_by_year, markers=True, dashes=False, 
    ax=ax, palette=[charts_colors['blue'], charts_colors['yellow']],
    markeredgecolor=bck_color, markeredgewidth=.5, markersize=7.5, label='ere')

ax.set_xticks(np.arange(2016, 2021, 1))
ax.set_xlabel("")

ax.set_title('Production Solaire (gW) & Rayonnement Solaire (kW/m2) par Année- Somme')
plt.legend(labels=['Solaire', 'Rayonnement Solaire'])

plt.savefig("outputs/03.3.3_-_Solaire_par_annee.png", dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

sns.boxenplot(data=solaire['solaire'], ax=ax, palette=[charts_colors['blue_dark']], linewidth=1, k_depth='tukey')

ax.set_ylabel('Production en gW')
ax.set_title('Distribution de la production solaire')

plt.savefig("outputs/03.3.4_-_Distribution_solaire.png", dpi=150, bbox_inches='tight')

plt.show()


In [ ]:
grp_rayonnement = solaire[solaire['annee'] == 2019].copy()
grp_rayonnement = grp_rayonnement.groupby(['region'])[['rayonnement_solaire']].sum().reset_index()
grp_rayonnement_sorted = grp_rayonnement.sort_values(by='rayonnement_solaire', ascending=True)


fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(
    x='region', y='rayonnement_solaire', data=grp_rayonnement_sorted,
    ax=ax, ci=False, palette=[charts_colors['yellow']], edgecolor=grid_color)

plt.setp(ax.get_xticklabels(), rotation=90)

ax.set_xlabel(None)
ax.set_ylabel('Rayonnement Solaire')

ax.set_title('Rayonnement solaire (kW/m2) selon les régions - Total en 2019')

plt.savefig("outputs/03.3.5_-_Rayonnement_solaire_par_region.png", dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
grp_solaire = solaire[solaire['annee'] == 2019].copy()
grp_solaire = grp_solaire.groupby(['region'])[['solaire']].sum().reset_index()
grp_solaire_sorted = grp_solaire.sort_values(by='solaire', ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(
    x='region', y='solaire', data=grp_solaire_sorted, 
    ax=ax, ci=False, palette=[charts_colors['yellow']], edgecolor=grid_color)
for tick in ax.get_xticklabels():
    tick.set_rotation(90)

ax.set_xlabel(None)
ax.set_ylabel('Rayonnement Solaire')

ax.set_title('Production solaire (gW) selon les régions - Total en 2019')

plt.savefig("outputs/03.3.6_-_Production_solaire_par_region.png", dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
c = mcolors.ColorConverter().to_rgb
cmap = pn.make_colormap(
    [c(charts_colors['yellow_light']), c(charts_colors['orange_dark'])])

In [ ]:
rs = solaire[ solaire['rayonnement_solaire'] > 0 ].copy()

rs = rs.groupby(['code_insee_region', 'region'])['rayonnement_solaire'].mean().reset_index()
gdf = pn.get_geometry(rs)

fig, ax = plt.subplots(figsize=(10, 10))

gdf.plot('rayonnement_solaire', edgecolor=bck_color, legend=True, cmap=cmap, legend_kwds={'shrink': 0.5}, ax=ax)
ax.set_title('Rayonnement Solaire')
ax.axis('off')

plt.savefig("outputs/03.3.6_-_Carte_rayonnement_solaire_par_region.png", dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
rs = solaire[ solaire['solaire'] > 0 ].copy()

rs = rs.groupby(['code_insee_region', 'region'])['solaire'].mean().reset_index()
gdf = pn.get_geometry(rs)

fig, ax = plt.subplots(figsize=(10, 10))

gdf.plot('solaire', edgecolor=bck_color, legend=True, cmap=cmap, legend_kwds={'shrink': 0.5}, ax=ax)
ax.set_title('Solaire')
ax.axis('off')
plt.savefig("outputs/03.3.6_-_Carte_production_solaire_par_region.png", dpi=150, bbox_inches='tight')

plt.show()